In [ ]:
# ! pip install --upgrade pip
# ! pip install --user dask[dataframe]
# ! pip install --user numpy scipy matplotlib seaborn boost_histogram pandas uproot awkward-pandas

In [ ]:
import uproot
import pandas as pd
import dask.dataframe as dd
import dask
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns

import sys

In [ ]:
sys.path.append("/home/belle2/amubarak/Ds2D0enue_Analysis/10-Python_Functions/")

# Prep-Work

### Import Data

In [ ]:
# Specify the branches (variables) to load
branches_to_load = ["e_electronID",
                    "Ds_D0_Dstarplus","Ds_D0_Dstar0","Ds_D0_NoDstarplusDstar0","Ds_D0_Other",
                    "D0_mcPDG","D0_isSignal",
                    "Ds_diff_D0pi","Ds_massDifference_0",
                    "Ds_ifNANgiveX_isSignal_5","Ds_mcPDG",
                    "Ds_extraInfo_BkgBDT","Ds_gammaveto_M_Correction"]  # Replace with actual branch names

In [ ]:
# Function to load data efficiently using uproot.iterate()
def load_data_with_iterate(file_path, tree_name, step_size="100 MB"):
    df_chunks = uproot.iterate(
        f"{file_path}:{tree_name}",
        library="pd",
        step_size=step_size,
        branches=branches_to_load
    )
    return pd.concat(df_chunks)

# Function to load data using uproot.iterate() with Dask
def load_data_with_dask(file_path, tree_name, step_size="100 MB"):
    df_chunks = uproot.iterate(
        f"{file_path}:{tree_name}",
        library="pd",
        step_size=step_size,
        branches=branches_to_load
    )
    ddf = dd.from_pandas(pd.concat(df_chunks), npartitions=10)  # Convert to Dask DataFrame
    return ddf.compute()  # Compute final DataFrame

In [ ]:
# Define paths
signal_file = "/home/belle2/amubarak/C01-Simulated_Events/Ds2D0enu-Signal.root"
background_path = "/home/belle2/amubarak/C03-Grid/Completed/Ds2D0e-Generic_Ds_"

tree_name = "Dstree"
Date = "0313"
Attempt = "1"
samples = ["Signal","All","ccbar"]
# samples = ["Signal", "All", "ccbar", "charged", "ddbar", "mixed", "ssbar", "uubar", "uds"]

# Load data using uproot.iterate() with Dask
DataFrames = {}
DataFrames[samples[0]] = load_data_with_dask(signal_file, tree_name)

for s in samples[1:]:
    file_path = f"{background_path}{Date}25_{Attempt}_{s}.root"
    DataFrames[s] = load_data_with_dask(file_path, tree_name)

In [ ]:
# # In this notebook we only process the main signal and the generic events,
# # for illustration purposes.
# # You can add other backgrounds after if you wish.
# # samples = ["Signal", "ccbar"]
# samples = ["Signal","All","ccbar"]

In [ ]:
# DataFrames = {}  # define empty dictionary to hold dataframes
# Date = "0313"
# Attempt = "1"

# # Signal:
# DataFrames[samples[0]] =  uproot.concatenate("/home/belle2/amubarak/C01-Simulated_Events/Ds2D0enu-Signal.root:Dstree",library='pd')
# # Background
# for s in samples[1:]: # loop over samples
#     DataFrames[s] =  uproot.concatenate("/home/belle2/amubarak/C03-Grid/Completed/Ds2D0e-Generic_Ds_" + Date +"25_"+ Attempt +"_"+ s +".root:Dstree",library='pd')

In [ ]:
DataFrames["BDT"] =  uproot.concatenate("/home/belle2/amubarak/C02-MVA/Completed/Ds2D0enu_FoM.root:variables",library='pd')

The line below is to look at the available variables.

In [ ]:
DataFrames["BDT"].columns.tolist()

In [ ]:
DataFrames["Signal"].columns.tolist()

### Setup
The code below will be used to apply cuts to the data.

In [ ]:
# Electron ID
#-------------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['e_electronID']>=0.95]
# DataFrames["ccbar"] = DataFrames["ccbar"][DataFrames["ccbar"]['e_electronID']>=0.95]
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_gammaveto_em_electronID']>=0.95]
# DataFrames["ccbar"] = DataFrames["ccbar"][DataFrames["ccbar"]['Ds_gammaveto_em_electronID']>=0.95]

# Photon Conversion
#-------------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=0.1]
# DataFrames["ccbar"] = DataFrames["ccbar"][DataFrames["ccbar"]['Ds_gammaveto_M_Correction']>=0.1]

# Peaking Background Removal
#----------------------------
# DataFrames["ccbar"] = DataFrames["ccbar"][(DataFrames["ccbar"]['Ds_diff_D0pi']>=0.15)]
# DataFrames["Signal"] = DataFrames["Signal"][(DataFrames["Signal"]['Ds_diff_D0pi']>=0.15)]

# # Vertex Fitting
# #----------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_chiProb']>=0.01]
# DataFrames["ccbar"] = DataFrames["ccbar"][DataFrames["ccbar"]['Ds_chiProb']>=0.01]

# Dalitz Removal
#----------------------------
# DataFrames["ccbar"] = DataFrames["ccbar"][(DataFrames["ccbar"]['Ds_pi0veto_M_Correction']<=0.08) | (DataFrames["ccbar"]['Ds_pi0veto_M_Correction']>=0.16)]
# DataFrames["Signal"] = DataFrames["Signal"][(DataFrames["Signal"]['Ds_pi0veto_M_Correction']<=0.08) | (DataFrames["Signal"]['Ds_pi0veto_M_Correction']>=0.16)]

# Vertex Fit
#----------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames[samples[0]]['Ds_chiProb_Ds_rank']==1]
# DataFrames["ccbar"] = DataFrames["ccbar"][DataFrames[samples[1]]['Ds_chiProb_Ds_rank']==1]

# # D0 Invariant Mass
# #-----------------------
# DataFrames["Signal"] = DataFrames["Signal"][(DataFrames["Signal"]['D0_dM']>=-0.02) & (DataFrames["Signal"]['D0_dM']<=0.02)]
# DataFrames["ccbar"] = DataFrames["ccbar"][(DataFrames["ccbar"]['D0_dM']>=-0.02) & (DataFrames["ccbar"]['D0_dM']<=0.02)]
# DataFrames["All"] = DataFrames["All"][(DataFrames["All"]['D0_dM']>=-0.02) & (DataFrames["All"]['D0_dM']<=0.02)]

In [ ]:
print(abs(DataFrames["ccbar"].query('D0_isSignal!=1')['D0_mcPDG']).value_counts(normalize=True,dropna=False).apply(lambda x: f"{x:.6f}"))

In [ ]:
print(abs(DataFrames["ccbar"][(DataFrames["ccbar"]['Ds_extraInfo_BkgBDT']>=0.531)]['Ds_mcPDG']).value_counts(dropna=False).apply(lambda x: f"{x:.6f}"))

In [ ]:
print(abs(DataFrames["ccbar"].query('D0_mcPDG==20413')['D0_mcErrors']).value_counts(normalize=False,dropna=False).apply(lambda x: f"{x:.6f}"))

# 1-D Histogram

In [ ]:
Bins=50
Density = False
Stacked = False
Range = [0.1,0.7]
perBin = ((Range[1] - Range[0])/Bins)*1000
var = 'Ds_diff_D0pi'
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$isSignal(D_s^{+})=1$'
label2= r'$isSignal(D_s^{+})=0$'
label3= r'$NaN$'

labels=[label1,label2,label3]
colors=['#fd7f6f','#7eb0d5','purple']

data = [DataFrames[samples[0]][DataFrames[samples[0]]['Ds_isSignal']==1][var],
        DataFrames[samples[0]][DataFrames[samples[0]]['Ds_isSignal']==0][var],
        DataFrames[samples[0]][DataFrames[samples[0]]['Ds_isSignal'].isna()][var]
       ]


plt.hist(data[::-1], color=colors[::-1], label=labels[::-1], alpha=1, range=Range, stacked=Stacked, density=Density, bins=Bins, histtype='step')

# Title
#---------
# Signal
plt.title(r'$2M\;Events$', loc = "left")
plt.title(r'$\bf Signal\;Events$', loc = "right")
# # Background
# plt.title(r'$\int\mathcal{L}dt\approx\;100$ fb$^{-1}$', loc = "left")
# plt.title(r'$\bf Generic\;c\bar{c}\;Events$', loc = "right")
# Label
#---------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
plt.yscale("log") 
plt.legend()
plt.show()

In [ ]:
# Stacked = True
# Density = False
# Bins = 50
# # i = 'Ds_diff_D0pi'
# i = 'Ds_massDifference_0'
# Range = [0.0, 0.25]
# BD = -1
# perBin = ((Range[1] - Range[0])/Bins)*1000
# print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

# label1= r'$c \bar{c}$'
# label2= r'$charged$'
# label3= r'$u \bar{u}, \; d \bar{d}, \;s \bar{s}$'
# label4= r'$mixed$'

# labels=[label1,label2,label3,label4]
# # colors=['C2','C1','C3','C4','C5']
# data=[DataFrames["ccbar"][(DataFrames["ccbar"]['Ds_extraInfo_BkgBDT']>=BD)][i],
#       DataFrames["charged"][(DataFrames["charged"]['Ds_extraInfo_BkgBDT']>=BD)][i],
#       DataFrames["uds"][(DataFrames["uds"]['Ds_extraInfo_BkgBDT']>=BD)][i],
#       DataFrames["mixed"][(DataFrames["mixed"]['Ds_extraInfo_BkgBDT']>=BD)][i],
#       ]

# factor = 1.5
# plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['Ds_extraInfo_BkgBDT']>=BD)][i], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['Ds_extraInfo_BkgBDT']>=BD)][i]), ls='--', linewidth=1.5)
# plt.hist(data[::-1], label=labels[::-1], density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
# # plt.axvspan(Range[0],0.15,color='gray',alpha=0.2)
# # plt.axvline(0.15,ls='--',color='gray')

# # Title
# #--------
# # plt.title(r'$BCS: Vertex \; Fit \; \chi^{2}$', loc = "left")
# plt.title(r'$D_s^{+} \rightarrow D^{0} e^{+} \nu_{e}$', loc = "left")
# plt.title(r'$\int\mathcal{L}dt\approx\;1443.999$ fb$^{-1}$', loc = "right")
# # Label
# #-------
# plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
# plt.xlabel(r'$\Delta m(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# # plt.yscale("log")
# # plt.xscale("log")
# # plt.ylim(0, 25000)
# plt.legend()
# plt.show()

In [ ]:
# Stacked = True
# Density = False
# Bins = 50
# i = 'Ds_diff_D0pi'
# # i = 'Ds_massDifference_0'
# Range = [0.1, 0.55]
# BD = -1
# perBin = ((Range[1] - Range[0])/Bins)*1000
# print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

# label1= r'$c \bar{c}$'
# label2= r'$charged$'
# label3= r'$u \bar{u}, \; d \bar{d}, \;s \bar{s}$'
# label4= r'$mixed$'

# labels=[label1,label2,label3,label4]
# # colors=['C2','C1','C3','C4','C5']
# data=[DataFrames["ccbar"][(DataFrames["ccbar"]['Ds_extraInfo_BkgBDT']>=BD)][i],
#       DataFrames["charged"][(DataFrames["charged"]['Ds_extraInfo_BkgBDT']>=BD)][i],
#       DataFrames["uds"][(DataFrames["uds"]['Ds_extraInfo_BkgBDT']>=BD)][i],
#       DataFrames["mixed"][(DataFrames["mixed"]['Ds_extraInfo_BkgBDT']>=BD)][i],
#       ]

# factor = 3
# plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['Ds_extraInfo_BkgBDT']>=BD)][i], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['Ds_extraInfo_BkgBDT']>=BD)][i]), ls='--', linewidth=1.5)
# plt.hist(data[::-1], label=labels[::-1], density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
# # plt.axvspan(Range[0],0.15,color='gray',alpha=0.2)
# # plt.axvline(0.15,ls='--',color='gray')

# # Title
# #--------
# # plt.title(r'$BCS: Vertex \; Fit \; \chi^{2}$', loc = "left")
# plt.title(r'$D_s^{+} \rightarrow D^{0} e^{+} \nu_{e}$', loc = "left")
# plt.title(r'$\int\mathcal{L}dt\approx\;1443.999$ fb$^{-1}$', loc = "right")
# # Label
# #-------
# plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
# plt.xlabel(r'$\Delta m(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# # plt.yscale("log")
# # plt.xscale("log")
# # plt.ylim(0, 25000)
# plt.legend()
# plt.show()

In [ ]:
Stacked = False
Density = False
Bins = 50
var = 'Ds_diff_D0pi'
# i = 'Ds_massDifference_0'
Range = [0.1, 0.55]
dM = -1
FD = -1
BS = 0.531
i = 0
Samples = "ccbar"
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$D^{*+} \rightarrow D^{0} \pi^{+}$'
label2= r'$D^{*0} \rightarrow D^{0} \gamma, D^{0} \pi^{0}$'
label3= r'$D^{0}$'
label4= r'$Other$'

labels=[label1,label2,label3,label4]
colors=['C2','C1','C4','C5']
data=[
      DataFrames[Samples][(abs(DataFrames[Samples]['Ds_D0_Dstarplus'])==1) & (abs(DataFrames[Samples]['D0_isSignal'])==1) & (DataFrames[Samples]['Ds_extraInfo_BkgBDT']>=BS)][var],
      DataFrames[Samples][(abs(DataFrames[Samples]['Ds_D0_Dstar0'])==1) & (abs(DataFrames[Samples]['D0_isSignal'])==1) & (DataFrames[Samples]['Ds_extraInfo_BkgBDT']>=BS)][var],
      DataFrames[Samples][(abs(DataFrames[Samples]['Ds_D0_NoDstarplusDstar0'])==1) & (abs(DataFrames[Samples]['D0_isSignal'])==1) & (DataFrames[Samples]['Ds_extraInfo_BkgBDT']>=BS)][var],
      DataFrames[Samples][((abs(DataFrames[Samples]['Ds_D0_Other'])==1) | ((abs(DataFrames[Samples]['D0_mcPDG'])==421) & (abs(DataFrames[Samples]['D0_isSignal'])==0))) & (DataFrames[Samples]['Ds_extraInfo_BkgBDT']>=BS)][var],
      ]

factor = 0.7
plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['Ds_extraInfo_BkgBDT']>=BS)][var], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['Ds_extraInfo_BkgBDT']>=BS)][var]), ls='--', linewidth=1.5)
plt.hist(data, color=colors, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
# plt.axvspan(Range[0],0.15,color='gray',alpha=0.2)
# plt.axvline(0.15,ls='--',color='gray')

# Title
#--------
plt.title(r'$BDT \geq 0.531$', loc = "left")
plt.title(r'$\int\mathcal{L}dt\approx\;1443.999$ fb$^{-1}$', loc = "right")
# Label
#-------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# plt.yscale("log")
# plt.xscale("log")
plt.ylim(0, 15000)
plt.legend()
plt.show()

In [ ]:
Stacked = True
Density = False
Bins = 50
i = 'Ds_diff_D0pi'
# i = 'Ds_massDifference_0'
Range = [0.1, 0.55]
Op = -1
dM = -1
Hits = -1
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$Other$'
label2= r'$Comb.$'
label3= r'$NaN$'
label4= r'$D^{*0}$'
label5= r'$D^{*+} \rightarrow D^{0} \pi^{+}$'

labels=[label1,label2,label3,label4,label5]
colors=['C2','C1','C3','C4','C5']
data=[DataFrames["All"][(abs(DataFrames["All"]["Ds_Other"])==1) & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM)][i],
      DataFrames["All"][(abs(DataFrames["All"]["Ds_Comb"])==1) & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM)][i],
      DataFrames["All"][(abs(DataFrames["All"]["Ds_Failed"])==1) & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM)][i],
      DataFrames["All"][(abs(DataFrames["All"]["Ds_Dstar0"])==1) & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM)][i],
      DataFrames["All"][(abs(DataFrames["All"]["Ds_Dstarplus"])==1) & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM)][i]
      ]

factor = 0.5
plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["Signal"]['e_nPXDHits']>Hits)][i], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=dM)][i]), ls='--', linewidth=1.5)
plt.hist(data, color=colors, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
# plt.axvspan(Range[0],0.15,color='gray',alpha=0.2)
# plt.axvline(0.15,ls='--',color='gray')

# Title
#--------
plt.title(r'$Electron \; with \; Pion \; Mass$', loc = "left")
# Label
#-------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# plt.yscale("log")
# plt.xscale("log")
# plt.ylim(0, 80000)
plt.legend(loc='upper right')
plt.show()

In [ ]:
Stacked = False
Density = False
Bins = 50
Range = [0.1, 0.55]
Op = 0.531
dM = -1
# i = 'e_cos_theta'
# i = 'Ds_vpho_CMS_daughterAngle'
i = 'Ds_diff_D0pi'
# i = 'Ds_chiProb_noIP'
# i = 'Ds_chiProb'
# i = 'Ds_extraInfo_FastBDT'
# i = 'D0_chiProb'
# i = 'Ds_Ds_starminusDs_M_Correction'
# i = "Ds_gammaveto_M_Correction"
# i = 'D0_chiProb'
# i = "Ds_L_diff"
# var = 'e_cos_theta'
# i = 'e_pt'
perBin = ((Range[1] - Range[0])/Bins)*1000
# perBin = ((Range[1] - Range[0])/Bins)
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$D^{*+} \rightarrow D^{0} \pi^{+}$'
label3= r'$D^{0}$'
label4= r'$Other$'

labels1=[label1,label3,label4]
colors1=['C1','C2','C3']
data1=[
      DataFrames["All"][(abs(DataFrames["All"]['Ds_D0_Dstarplus'])==1) & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["All"]['Ds_extraInfo_BkgBDT']>=Op)][i],
      DataFrames["All"][(abs(DataFrames["All"]['Ds_D0_NoDstarplusDstar0'])==1) & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["All"]['Ds_extraInfo_BkgBDT']>=Op)][i],
      DataFrames["All"][(abs(DataFrames["All"]['Ds_D0_Other'])==1) & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["All"]['Ds_extraInfo_BkgBDT']>=Op)][i],
      ]
labels2=[r'$D^{*0} \; (Comb.)$',r'$D^{*0} \; (Peak)$']
colors2=['C4','C5']
data2=[
      DataFrames["All"][(abs(DataFrames["All"]['Ds_mcPDG'])!=423) & (abs(DataFrames["All"]['Ds_D0_Dstar0'])==1) & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["All"]['Ds_extraInfo_BkgBDT']>=Op)][i],
      DataFrames["All"][(abs(DataFrames["All"]['Ds_mcPDG'])==423) & (abs(DataFrames["All"]['Ds_D0_Dstar0'])==1) & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["All"]['Ds_extraInfo_BkgBDT']>=Op)][i],
      ]

# factor = 0.1
# plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["Signal"]["Ds_BS"]>=Op)][i], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["Signal"]["Ds_BS"]>=Op)][i]), ls='--', linewidth=1.5)
plt.hist(data1, color=colors1, label=labels1, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
plt.hist(data2, color=colors2, label=labels2, density=Density, stacked=False, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
# plt.axvspan(Range[0],0.15,color='gray',alpha=0.2)
# plt.axvline(0.58,ls='--',color='gray')

# Title
#--------
plt.title(r'$BDT \; \geq 0.531$', loc = "left")
# Label
#-------
# plt.ylabel(r'$Entries/(\; {width:.2f}\;)$'.format(width = perBin))
# plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c)$'.format(width = perBin))
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
# plt.xlabel(r'$p_{t} (e^{+}) [GeV/c]$')
# plt.xlabel(r'$\Delta \theta(D_s^{+} \; K^{+/-}/K_{s}^{0}) \; [rad]$')
# plt.xlabel(r'$cos\theta \; (e^{+})$')
# plt.xlabel(r'$p-value(D^{0})$')
# plt.xlabel(r'$p-value(D_{s}^{+})$')
# plt.xlabel(r'$p-value_{IP}(D_{s}^{+})$')
# plt.xlabel(r'$Fake D^{0} Suppression(D^{0})$')
# plt.xlabel(r'$m(e_{sig}^{+} e_{ROE}^{-})\;[GeV/c^{2}]$')
# plt.xlabel(r'$p_{t} \; (e^{+})\;[GeV/c]$')
plt.xlabel(r'$\Delta m(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# plt.xlabel(r'$\Delta m(D_s^{*+} - D_{s}^{+})\;[GeV/c^{2}]$')
# plt.xlabel(r'$cos\theta \; (e^{+})$')
# plt.xlabel(r'$p-value(D^{0})$')
# plt.xlabel(r'$\mid \vec{x}_{D_{s}^{+}} - \vec{x}_{D^{0}} \mid \; [cm]$')
# plt.xlabel(r'$dz \; (e^{+}) \; [cm]$')
# plt.yscale("log")
# plt.xscale("log")
plt.ylim(0, 14000)
plt.legend()
plt.show()

In [ ]:
Stacked = False
Density = False
Bins = 50
Range = [0.1, 0.55]
Op = 0.531
dM = -1
# i = 'e_cos_theta'
# i = 'Ds_vpho_CMS_daughterAngle'
i = 'Ds_diff_D0pi'
# i = 'Ds_chiProb_noIP'
# i = 'Ds_chiProb'
# i = 'Ds_extraInfo_FastBDT'
# i = 'D0_chiProb'
# i = 'Ds_Ds_starminusDs_M_Correction'
# i = "Ds_gammaveto_M_Correction"
# i = 'D0_chiProb'
# i = "Ds_L_diff"
# var = 'e_cos_theta'
# i = 'e_pt'
perBin = ((Range[1] - Range[0])/Bins)*1000
# perBin = ((Range[1] - Range[0])/Bins)
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

# label1= r'$D^{*+} \rightarrow D^{0} \pi^{+}$'
# label3= r'$D^{0}$'
# label4= r'$Other$'

# labels1=[label1,label3,label4]
# colors1=['C1','C2','C3']
# data1=[
#       DataFrames["All"][(abs(DataFrames["All"]['Ds_D0_Dstarplus'])==1) & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["All"]['Ds_extraInfo_BkgBDT']>=Op)][i],
#       DataFrames["All"][(abs(DataFrames["All"]['Ds_D0_NoDstarplusDstar0'])==1) & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["All"]['Ds_extraInfo_BkgBDT']>=Op)][i],
#       DataFrames["All"][(abs(DataFrames["All"]['Ds_D0_Other'])==1) & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["All"]['Ds_extraInfo_BkgBDT']>=Op)][i],
#       ]
labels2=[r'$D^{*0} \; (Comb.)$',r'$D^{*0} \; (Peak)$']
colors2=['C4','C5']
data2=[
      DataFrames["All"][(abs(DataFrames["All"]['Ds_mcPDG'])!=423) & (abs(DataFrames["All"]['Ds_D0_Dstar0'])==1) & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["All"]['Ds_extraInfo_BkgBDT']>=Op)][i],
      DataFrames["All"][(abs(DataFrames["All"]['Ds_mcPDG'])==423) & (abs(DataFrames["All"]['Ds_D0_Dstar0'])==1) & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["All"]['Ds_extraInfo_BkgBDT']>=Op)][i],
      ]

factor = 0.1
plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["Signal"]['Ds_extraInfo_BkgBDT']>=Op)][i], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["Signal"]['Ds_extraInfo_BkgBDT']>=Op)][i]), ls='--', linewidth=1.5)
# plt.hist(data1, color=colors1, label=labels1, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
plt.hist(data2, color=colors2, label=labels2, density=Density, stacked=True, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
# plt.axvspan(Range[0],0.15,color='gray',alpha=0.2)
# plt.axvline(0.58,ls='--',color='gray')

# Title
#--------
plt.title(r'$BDT \; \geq 0.531$', loc = "left")
# Label
#-------
# plt.ylabel(r'$Entries/(\; {width:.2f}\;)$'.format(width = perBin))
# plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c)$'.format(width = perBin))
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
# plt.xlabel(r'$p_{t} (e^{+}) [GeV/c]$')
# plt.xlabel(r'$\Delta \theta(D_s^{+} \; K^{+/-}/K_{s}^{0}) \; [rad]$')
# plt.xlabel(r'$cos\theta \; (e^{+})$')
# plt.xlabel(r'$p-value(D^{0})$')
# plt.xlabel(r'$p-value(D_{s}^{+})$')
# plt.xlabel(r'$p-value_{IP}(D_{s}^{+})$')
# plt.xlabel(r'$Fake D^{0} Suppression(D^{0})$')
# plt.xlabel(r'$m(e_{sig}^{+} e_{ROE}^{-})\;[GeV/c^{2}]$')
# plt.xlabel(r'$p_{t} \; (e^{+})\;[GeV/c]$')
plt.xlabel(r'$\Delta m(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# plt.xlabel(r'$\Delta m(D_s^{*+} - D_{s}^{+})\;[GeV/c^{2}]$')
# plt.xlabel(r'$cos\theta \; (e^{+})$')
# plt.xlabel(r'$p-value(D^{0})$')
# plt.xlabel(r'$\mid \vec{x}_{D_{s}^{+}} - \vec{x}_{D^{0}} \mid \; [cm]$')
# plt.xlabel(r'$dz \; (e^{+}) \; [cm]$')
# plt.yscale("log")
# plt.xscale("log")
plt.ylim(0, 8000)
plt.legend()
plt.show()

# Reverse Cut
Revering the cuts to get the shapes

In [ ]:
Stacked = False
Density = False
Bins = 50
var = 'e_electronID'
# i = 'Ds_massDifference_0'
Range = [0., 1.0]
BS = 2
i = 0
Samples = "ccbar"
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$D^{*+} \rightarrow D^{0} \pi^{+}$'
label2= r'$D^{*0} \rightarrow D^{0} \gamma, D^{0} \pi^{0}$'
label3= r'$D^{0}$'
label4= r'$Other$'

labels=[label1,label2,label3,label4]
colors=['C2','C1','C4','C5']
data=[
      DataFrames[Samples][(abs(DataFrames[Samples]['Ds_D0_Dstarplus'])==1) & (abs(DataFrames[Samples]['D0_isSignal'])==1) & (DataFrames[Samples]['e_electronID']<=BS)][var],
      DataFrames[Samples][(abs(DataFrames[Samples]['Ds_D0_Dstar0'])==1) & (abs(DataFrames[Samples]['D0_isSignal'])==1) & (DataFrames[Samples]['e_electronID']<=BS)][var],
      DataFrames[Samples][(abs(DataFrames[Samples]['Ds_D0_NoDstarplusDstar0'])==1) & (abs(DataFrames[Samples]['D0_isSignal'])==1) & (DataFrames[Samples]['e_electronID']<=BS)][var],
      DataFrames[Samples][((abs(DataFrames[Samples]['Ds_D0_Other'])==1) | ((abs(DataFrames[Samples]['D0_mcPDG'])==421) & (abs(DataFrames[Samples]['D0_isSignal'])==0))) & (DataFrames[Samples]['e_electronID']<=BS)][var],
      ]

factor = 0.7
plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['e_electronID']<=BS)][var], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['e_electronID']<=BS)][var]), ls='--', linewidth=1.5)
plt.hist(data, color=colors, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
plt.axvspan(0.5,Range[1],color='gray',alpha=0.2)
plt.axvline(0.5,ls='--',color='gray')

# Title
#--------
# plt.title(r'$BDT \geq 0.531$', loc = "left")
plt.title(r'$\int\mathcal{L}dt\approx\;200$ fb$^{-1}$', loc = "right")
# Label
#-------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$electronID\;(e^{+})$')
# plt.yscale("log")
# plt.xscale("log")
# plt.ylim(0, 15000)
plt.legend()
plt.show()

In [ ]:
Stacked = False
Density = False
Bins = 50
# var = 'e_electronID'
var = 'Ds_diff_D0pi'
# i = 'Ds_massDifference_0'
Range = [0.1, 0.55]
BS = 0.5
i = 0
Samples = "ccbar"
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$D^{*+} \rightarrow D^{0} \pi^{+}$'
label2= r'$D^{*0} \rightarrow D^{0} \gamma, D^{0} \pi^{0}$'
label3= r'$D^{0}$'
label4= r'$Other$'

labels=[label1,label2,label3,label4]
colors=['C2','C1','C4','C3']
data=[
      DataFrames[Samples][(abs(DataFrames[Samples]['Ds_D0_Dstarplus'])==1) & (abs(DataFrames[Samples]['D0_isSignal'])==1) & (DataFrames[Samples]['e_electronID']<=BS)][var],
      DataFrames[Samples][(abs(DataFrames[Samples]['Ds_D0_Dstar0'])==1) & (abs(DataFrames[Samples]['D0_isSignal'])==1) & (DataFrames[Samples]['e_electronID']<=BS)][var],
      DataFrames[Samples][(abs(DataFrames[Samples]['Ds_D0_NoDstarplusDstar0'])==1) & (abs(DataFrames[Samples]['D0_isSignal'])==1) & (DataFrames[Samples]['e_electronID']<=BS)][var],
      DataFrames[Samples][((abs(DataFrames[Samples]['Ds_D0_Other'])==1) | ((abs(DataFrames[Samples]['D0_mcPDG'])==421) & (abs(DataFrames[Samples]['D0_isSignal'])==0))) & (DataFrames[Samples]['e_electronID']<=BS)][var],
      ]

factor = 0.7
plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['e_electronID']<=BS)][var], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['e_electronID']<=BS)][var]), ls='--', linewidth=1.5)
plt.hist(data, color=colors, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
# plt.axvspan(0.5,Range[0],color='gray',alpha=0.2)
# plt.axvline(0.5,ls='--',color='gray')

# Title
#--------
plt.title(r'$electronID \leq 0.5$', loc = "left")
plt.title(r'$\int\mathcal{L}dt\approx\;200$ fb$^{-1}$', loc = "right")
# Label
#-------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# plt.yscale("log")
# plt.xscale("log")
plt.ylim(0, 40000)
plt.legend()
plt.show()

In [ ]:
Stacked = False
Density = False
Bins = 50
# var = 'e_electronID'
var = 'Ds_diff_D0pi'
# i = 'Ds_massDifference_0'
Range = [0.1, 0.55]
BS = 0.5
i = 0
Samples = "ccbar"
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$D^{*+} \rightarrow D^{0} \pi^{+}$'
label2= r'$D^{*0} \rightarrow D^{0} \gamma, D^{0} \pi^{0}$'
label3= r'$D^{0}$'
label4= r'$Other$'

labels=[label1,label2,label3,label4]
colors=['C2','C1','C4','C5']
data=[
      DataFrames[Samples][(abs(DataFrames[Samples]['Ds_D0_Dstarplus'])==1) & (abs(DataFrames[Samples]['D0_isSignal'])==1) & (DataFrames[Samples]['e_electronID']<=BS)][var],
      DataFrames[Samples][(abs(DataFrames[Samples]['Ds_D0_Dstar0'])==1) & (abs(DataFrames[Samples]['D0_isSignal'])==1) & (DataFrames[Samples]['e_electronID']<=BS)][var],
      DataFrames[Samples][(abs(DataFrames[Samples]['Ds_D0_NoDstarplusDstar0'])==1) & (abs(DataFrames[Samples]['D0_isSignal'])==1) & (DataFrames[Samples]['e_electronID']<=BS)][var],
      DataFrames[Samples][((abs(DataFrames[Samples]['Ds_D0_Other'])==1) | ((abs(DataFrames[Samples]['D0_mcPDG'])==421) & (abs(DataFrames[Samples]['D0_isSignal'])==0))) & (DataFrames[Samples]['e_electronID']<=BS)][var],
      ]

factor = 0.7
plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['e_electronID']<=BS)][var], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['e_electronID']<=BS)][var]), ls='--', linewidth=1.5)
plt.hist(data, color=colors, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
# plt.axvspan(0.5,Range[0],color='gray',alpha=0.2)
# plt.axvline(0.5,ls='--',color='gray')

# Title
#--------
plt.title(r'$BDT \geq 0.531$', loc = "left")
plt.title(r'$\int\mathcal{L}dt\approx\;200$ fb$^{-1}$', loc = "right")
# Label
#-------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# plt.yscale("log")
# plt.xscale("log")
plt.ylim(0, 40000)
plt.legend()
plt.show()

In [ ]:
Stacked = False
Density = False
Bins = 50
var = 'Ds_gammaveto_M_Correction'
# i = 'Ds_massDifference_0'
Range = [0., 0.5]
dM = 2
BS = 0.5
i = 0
Samples = "ccbar"
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$D^{*+} \rightarrow D^{0} \pi^{+}$'
label2= r'$D^{*0} \rightarrow D^{0} \gamma, D^{0} \pi^{0}$'
label3= r'$D^{0}$'
label4= r'$Other$'

labels=[label1,label2,label3,label4]
colors=['C2','C1','C4','C5']
data=[
      DataFrames[Samples][(abs(DataFrames[Samples]['Ds_D0_Dstarplus'])==1) & (abs(DataFrames[Samples]['D0_isSignal'])==1) & (DataFrames[Samples]['Ds_gammaveto_M_Correction']<=dM) & (DataFrames[Samples]['e_electronID']>=BS)][var],
      DataFrames[Samples][(abs(DataFrames[Samples]['Ds_D0_Dstar0'])==1) & (abs(DataFrames[Samples]['D0_isSignal'])==1) & (DataFrames[Samples]['Ds_gammaveto_M_Correction']<=dM) & (DataFrames[Samples]['e_electronID']>=BS)][var],
      DataFrames[Samples][(abs(DataFrames[Samples]['Ds_D0_NoDstarplusDstar0'])==1) & (abs(DataFrames[Samples]['D0_isSignal'])==1) & (DataFrames[Samples]['Ds_gammaveto_M_Correction']<=dM) & (DataFrames[Samples]['e_electronID']>=BS)][var],
      DataFrames[Samples][((abs(DataFrames[Samples]['Ds_D0_Other'])==1) | ((abs(DataFrames[Samples]['D0_mcPDG'])==421) & (abs(DataFrames[Samples]['D0_isSignal'])==0))) & (DataFrames[Samples]['Ds_gammaveto_M_Correction']<=dM) & (DataFrames[Samples]['e_electronID']>=BS)][var],
      ]

factor = 0.7
plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']<=dM) & (DataFrames["Signal"]['e_electronID']>=BS)][var], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']<=dM) & (DataFrames["Signal"]['e_electronID']>=BS)][var]), ls='--', linewidth=1.5)
plt.hist(data, color=colors, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
plt.axvspan(0.05,Range[1],color='gray',alpha=0.2)
plt.axvline(0.05,ls='--',color='gray')

# Title
#--------
# plt.title(r'$BDT \geq 0.531$', loc = "left")
plt.title(r'$\int\mathcal{L}dt\approx\;200$ fb$^{-1}$', loc = "right")
# Label
#-------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$m(e^{-} e^{+})\;[GeV/c^{2}]$')
# plt.yscale("log")
# plt.xscale("log")
# plt.ylim(0, 15000)
plt.legend()
plt.show()

In [ ]:
Stacked = False
Density = False
Bins = 50
var = 'Ds_diff_D0pi'
# var = 'Ds_gammaveto_M_Correction'
# i = 'Ds_massDifference_0'
Range = [0.1, 0.55]
dM = 0.05
i = 0
Samples = "All"
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$D^{*+} \rightarrow D^{0} \pi^{+}$'
label2= r'$D^{*0} \rightarrow D^{0} \gamma, D^{0} \pi^{0}$'
label3= r'$D^{0}$'
label4= r'$Other$'

labels=[label1,label2,label3,label4]
colors=['C2','C1','C4','C5']
data=[
      DataFrames[Samples][(abs(DataFrames[Samples]['Ds_D0_Dstarplus'])==1) & (abs(DataFrames[Samples]['D0_isSignal'])==1) & (DataFrames[Samples]['Ds_gammaveto_M_Correction']<=dM) & (DataFrames[Samples]['e_electronID']>=BS)][var],
      DataFrames[Samples][(abs(DataFrames[Samples]['Ds_D0_Dstar0'])==1) & (abs(DataFrames[Samples]['D0_isSignal'])==1) & (DataFrames[Samples]['Ds_gammaveto_M_Correction']<=dM) & (DataFrames[Samples]['e_electronID']>=BS)][var],
      DataFrames[Samples][(abs(DataFrames[Samples]['Ds_D0_NoDstarplusDstar0'])==1) & (abs(DataFrames[Samples]['D0_isSignal'])==1) & (DataFrames[Samples]['Ds_gammaveto_M_Correction']<=dM) & (DataFrames[Samples]['e_electronID']>=BS)][var],
      DataFrames[Samples][((abs(DataFrames[Samples]['Ds_D0_Other'])==1) | ((abs(DataFrames[Samples]['D0_mcPDG'])==421) & (abs(DataFrames[Samples]['D0_isSignal'])==0))) & (DataFrames[Samples]['Ds_gammaveto_M_Correction']<=dM) & (DataFrames[Samples]['e_electronID']>=BS)][var],
      ]

factor = 0.7
plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']<=dM) & (DataFrames["Signal"]['e_electronID']>=BS)][var], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']<=dM) & (DataFrames["Signal"]['e_electronID']>=BS)][var]), ls='--', linewidth=1.5)
plt.hist(data, color=colors, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
# plt.axvspan(0.1,Range[1],color='gray',alpha=0.2)
# plt.axvline(0.1,ls='--',color='gray')

# Title
#--------
plt.title(r'$m(e^{-} e^{+}) \leq 0.05 \; GeV/c^{2}$', loc = "left")
plt.title(r'$\int\mathcal{L}dt\approx\;200$ fb$^{-1}$', loc = "right")
# Label
#-------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# plt.yscale("log")
# plt.xscale("log")
# plt.ylim(0, 15000)
plt.legend()
plt.show()

In [ ]:
Stacked = False
Density = True
Bins = 50
var = 'Ds_diff_D0pi'
# var = 'Ds_gammaveto_M_Correction'
# i = 'Ds_massDifference_0'
Range = [0.1, 0.3]
dM = 0.1
i = 0
Samples = "All"
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$D^{*+} \rightarrow D^{0} \pi^{+}$'
label2= r'$D^{*0} \rightarrow D^{0} \gamma, D^{0} \pi^{0}$'
label3= r'$D^{0}$'
label4= r'$Other$'

labels=[label1,label2]
colors=['C2','C1']
data=[
      DataFrames[Samples][(abs(DataFrames[Samples]['Ds_D0_Dstar0'])==1) & (abs(DataFrames[Samples]['Ds_mcPDG'])==423) & (abs(DataFrames[Samples]['D0_isSignal'])==1) & (DataFrames[Samples]['Ds_gammaveto_M_Correction']<=dM)][var],
      DataFrames[Samples][(abs(DataFrames[Samples]['Ds_D0_Dstar0'])==1) & (abs(DataFrames[Samples]['Ds_mcPDG'])==423) & (abs(DataFrames[Samples]['D0_isSignal'])==1)][var],
      ]

factor = 0.7
plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']<=dM)][var], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']<=dM)][var]), ls='--', linewidth=1.5)
plt.hist(data, color=colors, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
# plt.axvspan(0.1,Range[1],color='gray',alpha=0.2)
# plt.axvline(0.1,ls='--',color='gray')

# Title
#--------
# plt.title(r'$BDT \geq 0.531$', loc = "left")
plt.title(r'$\int\mathcal{L}dt\approx\;200$ fb$^{-1}$', loc = "right")
# Label
#-------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# plt.yscale("log")
# plt.xscale("log")
# plt.ylim(0, 15000)
plt.legend()
plt.show()

# BDT Check

In [ ]:
from Functions import get_df_sig_bkg,get_nsig_nbkg,plot_var_sig_bkg,optimize_SB,plot_save
# "Ds_extraInfo_BkgBDT"
# "Ds_BS"
optimize_SB(df_sig=DataFrames["BDT"][DataFrames["BDT"]['C02__miMVA__slCompleted__slMVAFastBDT__ptxml_Ds_isSignal']==1],
            df_bkg=DataFrames["BDT"][DataFrames["BDT"]['C02__miMVA__slCompleted__slMVAFastBDT__ptxml_Ds_isSignal']==0],
            var='C02__miMVA__slCompleted__slMVAFastBDT__ptxml',
            Signal=DataFrames["BDT"][DataFrames["BDT"]['C02__miMVA__slCompleted__slMVAFastBDT__ptxml_Ds_isSignal']==1],
            Background=DataFrames["BDT"][DataFrames["BDT"]['C02__miMVA__slCompleted__slMVAFastBDT__ptxml_Ds_isSignal']==0],
            FoM='C02__miMVA__slCompleted__slMVAFastBDT__ptxml',
            xlabel='Classifier Output',
            Bins=50,Range=[0,1],
            varmin=0,varmax=0.93,select='right',Width=False)

In [ ]:
i = 0.531
print(abs(DataFrames["All"][(abs(DataFrames["All"]["Ds_extraInfo_BkgBDT"])>=-1)]['D0_isSignal']).value_counts(normalize=False,dropna=False).head(100))
print(abs(DataFrames["All"][(abs(DataFrames["All"]["Ds_extraInfo_BkgBDT"])>=i)]['D0_isSignal']).value_counts(normalize=False,dropna=False).head(100))

### Electron ID

In [ ]:
# Bins=50
# range_M=[0, 1]
# perBin = ((range_M[1] - range_M[0])/Bins)*1000
# print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

# plt.hist(df[df['e_isSignal']==0]['e_electronID'], label=r'$isSignal=0$', color="#009E73", alpha=0.9, bins=Bins, range=range_M,histtype='step')
# plt.hist(df[df['e_isSignal']==1]['e_electronID'], label=r'$isSignal=1$', color="#0072B2", alpha=0.9, bins=Bins, range=range_M,histtype='step')
# plt.title(r'$50k\;Events$', loc = "left")
# plt.title(r'$\boldsymbol{Signal\;Events}$', loc = "right")
# plt.ylabel(r'$Entries/(\; {width:.2f} \;MeV/c^2)$'.format(width = perBin))
# plt.xlabel(r'$electronID\;(e^{+})$')
# plt.legend()
# plt.show()